# Generate Interferograms Using OPERA-CSLCs Downloaded from ASF

Based on OPERA_Applications notebook: [Create_Interferogram_by_Streaming_CSLC-S1](https://github.com/OPERA-Cal-Val/OPERA_Applications/blob/main/CSLC/Discover/Create_Interferogram_by_Streaming_CSLC-S1.ipynb)

---

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-CSLC_Recipe_Book/issues">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-CSLC_Recipe_Book/issues">opensarlab_OPERA-CSLC_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, OPERA CSLCs, or ASF data access?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software

In [ ]:
%matplotlib inline
from getpass import getpass
from pathlib import Path
import re

import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
import shapely.wkt as wkt

import asf_search as disco

---
## 1. Create a directory to hold the downloaded data

The directory you create will be located in your home directory.

In [ ]:
data_dir = Path.home() / 'data'
data_dir.mkdir(exist_ok=True)
print(f'data_dir: {data_dir}')

---
## 2. Authenticate with `asf_search`

**Gather credentials for authentication with Earth Data Login**

In [ ]:
username = input('Enter your EDL username')
password = getpass('Enter your EDL password')

**Start an asf_search session**

In [ ]:
try:
    user_pass_session = disco.ASFSession().auth_with_creds(username, password)
except disco.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

---
# 3. Download the data

**Search and download data**

We'll be downloading data from the [2019 Ridgecrest Earthquake](https://en.wikipedia.org/wiki/2019_Ridgecrest_earthquakes).

In [ ]:
granule_list = [
    'OPERA_L2_CSLC-S1_T071-151219-IW2_20190704T135209Z_20240429T174024Z_S1A_VV_v1.1',
    'OPERA_L2_CSLC-S1_T071-151219-IW2_20190716T135210Z_20240430T020420Z_S1A_VV_v1.1',
    ]
results = disco.granule_search(granule_list)

for burst in results:
    burst.download(data_dir, session=user_pass_session)

bursts = list(data_dir.glob('*.h5'))
bursts.sort()

---
## 4. Create a list of CSLC acquisition dates

In [ ]:
date_regex = r"\d{8}T\d{6}Z(?=_\d{8}T\d{6}Z)"
dates = []
try:
    for b in bursts:
        dates.append(re.search(date_regex, str(b)).group(0))
    dates = [d[:8] for d in dates]
    print(dates)  
except AttributeError:
    raise Exception(f'Date string not found in {b}') 

---
## 5. Load the data with `h5py`

In [ ]:
# Get the burst ID
burst_id_regex = r'(?<=OPERA_L2_CSLC-S1_)T\d{3}-\d{6}-IW\d'
try:
    burst_id = re.search(burst_id_regex, str(bursts[0])).group(0)
except AttributeError:
    raise Exception(f'Burst ID not found in {str(bursts[0])}') 

# Load the reference CSLC
with h5py.File(bursts[0], 'r') as h5:
    bounding_polygon = h5['identification/bounding_polygon'][()].astype(str) 
    cslc_poly = wkt.loads(bounding_polygon)
    bbox = [cslc_poly.bounds[0], cslc_poly.bounds[2], cslc_poly.bounds[1], cslc_poly.bounds[3]]
    cslc_0 = h5['data/VV'][:]
    
# Load the secondary CSLC
with h5py.File(bursts[1], 'r') as h5:
    cslc_1 = h5[f'data/VV'][:]

---
## 6. Generate and plot the interferogram

**Calculate the interferogram**

In [ ]:
ifg = cslc_0 * np.conj(cslc_1)

**Plot the interferogram**

In [ ]:
# Convert each pixel to RGB, adjusting colorscale relative to data range
def colorize(array=[], cmap='RdBu', cmin=[], cmax=[]):
    normed_data = (array - cmin) / (cmax - cmin)    
    cm = matplotlib.colormaps.get_cmap(cmap)
    return cm(normed_data) 

def plot_ifg(data, bbox, title):
    fig, ax = plt.subplots(figsize=(10,3))
    cax = ax.imshow(colorize(data, 'jet', -np.pi, np.pi), cmap='jet',interpolation='nearest', origin='upper',extent=bbox, vmin=-np.pi, vmax=np.pi)
    cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
    cbar.set_ticks([-np.pi, 0., np.pi])
    cbar.set_ticklabels([r'$-\pi$', '$0$', r'$\pi$'])
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(title,fontsize=12)

plot_ifg(np.angle(ifg), bbox, f'IFG_{dates[1]}-{dates[0]}_{burst_id}')

---
## 7. Estimate and plot the coherence

**Estimate coherence**

In [ ]:
from scipy.ndimage import uniform_filter


def estimate_coherence_from_ifg(ifg, window_size):
    nan_mask = np.isnan(ifg)
    zero_mask = ifg == 0

    # Normalize to unit magnitude
    inp = np.exp(1j * np.nan_to_num(np.angle(ifg)))

    # The clipping is from possible partial windows producing correlation above 1
    cor = np.clip(np.abs(uniform_filter(inp, window_size)), 0, 1)

    # Return the input nans to nan
    cor[nan_mask] = np.nan

    # If the input was 0, the correlation is 0
    cor[zero_mask] = 0
    return cor


coh = estimate_coherence_from_ifg(ifg, 12)

**Plot the coherence**

In [ ]:
def plot_coh(data, bbox, title):
    avg_corr = np.nanmean(data)
    fig, ax = plt.subplots(figsize=(10,3))
    cax = ax.imshow(data, cmap='gray', interpolation='nearest', origin='upper', extent=bbox, vmin=0, vmax=1)
    cbar = fig.colorbar(cax, orientation='vertical', fraction=0.01, pad=0.02)
    cbar.set_ticks([0, 1])
    cbar.set_ticklabels(['0', '1'])
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'{title}, AVG: {avg_corr:.2f}',fontsize=12)


plot_coh(coh, bbox, f'COH_{dates[1]}-{dates[0]}_{burst_id}')

---
## 8. Multilook the interferogram and coherence data

**Multilook the interferogram**

In [ ]:
def multilook(input_array, rows_looks, cols_looks):
    # Get the dimensions of the input array
    nrows, ncols = input_array.shape
    
    # Calculate the dimensions of the output array
    nrows_looked = nrows // rows_looks
    ncols_looked = ncols // cols_looks

    # Initialize the output arrays
    temp_output = np.zeros((nrows, ncols_looked), dtype=np.complex64)
    output = np.zeros((nrows_looked, ncols_looked), dtype=np.complex64)

    # First pass: downsample columns
    for kk in range(nrows * ncols_looked):
        line = kk // ncols_looked
        col = kk % ncols_looked
        sum_val = input_array[line, col * cols_looks: (col + 1) * cols_looks].sum()
        temp_output[line, col] = sum_val

    # Second pass: downsample rows
    for kk in range(nrows_looked * ncols_looked):
        line = kk // ncols_looked
        col = kk % ncols_looked
        sum_val = temp_output[line * rows_looks:(line + 1) * rows_looks, col].sum()
        output[line, col] = sum_val

    # Normalize the output
    output /= (cols_looks * rows_looks)
    return output


ifg_multilooked = multilook(ifg, 8, 4)

**Plot the multilooked interferogram**

In [ ]:
plot_ifg(np.angle(ifg_multilooked), bbox, f'IFG_MULTILOOKED_{dates[1]}-{dates[0]}_{burst_id}')

**Estimate the multilooked coherence**

In [ ]:
coh_multilooked = estimate_coherence_from_ifg(ifg_multilooked, 3)

**Plot the multilooked coherence**

In [ ]:
plot_coh(coh_multilooked, bbox, f'COH_MULTILOOKED_{dates[1]}-{dates[0]}_{burst_id}')

As you can see, coherence increases significnatly after multilooking. See [Touzi et. al, 1996](https://doi.org/10.1109/IGARSS.1996.516435) for more information.

---
## 9. View all plots together

In [ ]:
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.widgets import RadioButtons

def pick_plot(plot_name):
    if plot_name == 'Interferogram (multilooked)':
        cax = ax['main'].imshow(colorize(np.angle(ifg_multilooked), 'jet', -np.pi, np.pi), cmap='jet', interpolation='nearest', origin='upper', extent=bbox, vmin=-np.pi, vmax=np.pi)
    elif plot_name == 'Interferogram':
        cax = ax['main'].imshow(colorize(np.angle(ifg[::4,::8]), 'jet', -np.pi, np.pi), cmap='jet', interpolation='nearest', origin='upper', extent=bbox, vmin=-np.pi, vmax=np.pi)
    elif plot_name == 'Coherence (multilooked)':
        cax = ax['main'].imshow(coh_multilooked, cmap='gray', interpolation='nearest', origin='upper', extent=bbox, vmin=0, vmax=1, aspect='equal')
    elif plot_name == 'Coherence':
        cax = ax['main'].imshow(coh[::4,::8], cmap='gray', interpolation='nearest', origin='upper', extent=bbox, vmin=0, vmax=1, aspect='equal')


fig, ax = plt.subplot_mosaic([['main', 'plots']], width_ratios=[5, 2], layout='constrained', figsize=(12,3))
cax = ax['main'].imshow(colorize(np.angle(ifg_multilooked), 'jet', -np.pi, np.pi), cmap='jet',interpolation='nearest', origin='upper',extent=bbox, vmin=-np.pi, vmax=np.pi)
cbar_coh = fig.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(0, 1), cmap='gray'), ax=ax['main'], orientation='vertical', fraction=0.01, pad=0.02, ticks=[0,1])
cbar_ifg = fig.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(-np.pi, np.pi), cmap='jet'), ax=ax['main'], orientation='vertical', fraction=0.01, pad=0.02, ticks=[-np.pi, 0, np.pi])
cbar_ifg.set_ticklabels([r'$-\pi$', '$0$', r'$\pi$'])

ax['main'].set_xlabel('Longitude')
ax['main'].set_ylabel('Latitude')
ax['main'].set_title('Main Plot', fontsize=12)

ax['plots'].set_title(f'Select Figure:', fontsize=12)
ax['plots'].set_facecolor('white')
radio = RadioButtons(ax['plots'], ('Interferogram (multilooked)', 'Interferogram', 'Coherence (multilooked)', 'Coherence'), radio_props={'s': 150})
radio.on_clicked(pick_plot)